This is for the project for Applied Data Science Capstone. I am going to predict the severity of an accident.  

# 1. Introduction/Business Problem 

Nowadays, with more and more vehicles on the road and people’s increasing needs for traveling. The traffic problems are becoming one of the biggest headaches in our daily life, such as traffic jam, accidents... Especially, in the several weather conditions, people who need to travel would really like a predict app that can predict/warm the possibility of car accident and its severity to improve their safety and avoid unnecessary troubles.. The app can use the weather/ road/ light information from the users to predict the accidents. With this app, drives could plan their trip ahead, and make the best decisions about the date/route... for their trip.  


# 2. Data 

I am going to use the provided sample data, since it has 194673 accident records and 37 attributes, providing a comprehensive dataset to train and test the model. The dataset has a severitycode column, labeling the accidents’ severity. It can be used in a supervised machine learning model. Location of the accidents, road condition, weather condition, junction type, car speeding, number of people involved, light condition and number of vehicles involved in the accidents, are major predictors for the probability and severity of the car accidents. There are 58188 type 1 severity cases and 136485 type 2 severity cases, so the data needs to be balanced, transformed, cleaned before training and testing.


In [1]:
import numpy as np 
import pandas as pd
import pylab as pl
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline 
import matplotlib.pyplot as plt

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [2]:
df=pd.read_csv('C:\My research\python machine learning\capstone project\Data-Collisions.csv')

C:\Users\lbq_h\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [4]:
df=df[['OBJECTID','ADDRTYPE','PERSONCOUNT','VEHCOUNT','JUNCTIONTYPE','WEATHER','ROADCOND','LIGHTCOND','SPEEDING','SEVERITYCODE']]

In [5]:
df.head()

,OBJECTID,ADDRTYPE,PERSONCOUNT,VEHCOUNT,JUNCTIONTYPE,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,SEVERITYCODE
0,1,Intersection,2,2,At Intersection (intersection related),Overcast,Wet,Daylight,NaN,2
1,2,Block,2,2,Mid-Block (not related to intersection),Raining,Wet,Dark - Street Lights On,NaN,1
2,3,Block,4,3,Mid-Block (not related to intersection),Overcast,Dry,Daylight,NaN,1
3,4,Block,3,3,Mid-Block (not related to intersection),Clear,Dry,Daylight,NaN,1
4,5,Intersection,2,2,At Intersection (intersection related),Raining,Wet,Daylight,NaN,2


In [7]:
df.shape

(194673, 10)